In [24]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [25]:
# Import all required raw data files, I will ignore earnings and stock info from euronext now
# To make things easier, my index will always we a combination of stock_isin and date
balance_sheet_yahoo = pd.read_csv(r'raw_data\balance_sheet_yahoo.csv', sep='|').set_index(['stock_isin','date' ])
cashflow_yahoo = pd.read_csv(r'raw_data\cashflow_yahoo.csv', sep='|').set_index(['stock_isin','date' ])
financials_yahoo = pd.read_csv(r'raw_data\financials_yahoo.csv', sep='|').set_index(['stock_isin','date' ])
price_yahoo = pd.read_csv(r'raw_data\price_yahoo.csv', sep='|').set_index(['stock_isin','date' ])
stock_info_yahoo = pd.read_csv(
    r'raw_data\stock_info_yahoo.csv', sep='|').set_index(['stock_isin'])
stock_info_euronext = pd.read_csv(
    r'raw_data\stock_info_euronext.csv', sep='|').set_index(['stock_isin'])


In [26]:
# The rows that make up a distinct combination of stock_isin and date (year-end) will make up by full dataset
df = financials_yahoo.index.unique().to_frame()
df.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5552 entries, ('BMG9156K1018', '2021-12-31') to ('NO0010934748', '2019-12-31')
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stock_isin  5552 non-null   object
 1   date        5552 non-null   object
dtypes: object(2)
memory usage: 157.3+ KB


In [28]:
# We first focus on finding our predictor which which will be the one year holding return following the financial year end
# I want to focus on total return which will include dividends, this means days that pay a dividend get a higher return
price_yahoo['close_price_plus_dividends'] = price_yahoo['close_price'] + \
    price_yahoo['dividends']
price_yahoo.head()


open_price  high_price  low_price  close_price  \
stock_isin   date                                                         
BMG9156K1018 2019-07-12   83.686412   86.373593  79.655647    79.655647   
             2019-07-15   79.655642   80.596158  77.736229    77.736229   
             2019-07-16   77.736229   77.736229  76.776522    77.736229   
             2019-07-17   77.736229   77.736229  77.265975    77.736229   
             2019-07-18   76.786127   76.786127  73.907007    76.776527   

                         trading_volume  dividends  stock_splits  \
stock_isin   date                                                  
BMG9156K1018 2019-07-12         40831.0        0.0           0.0   
             2019-07-15          8810.0        0.0           0.0   
             2019-07-16         21400.0        0.0           0.0   
             2019-07-17         12966.0        0.0           0.0   
             2019-07-18          4789.0        0.0           0.0   

                         adjusted_close_price  close_price_plus_dividends  
stock_isin   date                                                          
BMG9156K1018 2019-07-12                   NaN                   79.655647  
             2019-07-15                   NaN                   77.736229  
             2019-07-16                   NaN                   77.736229  
             2019-07-17                   NaN                   77.736229  
             2019-07-18                   NaN                   76.776527

In [30]:
# First I will sort by date for each unique ISN
price_yahoo.sort_values(["stock_isin", "date"],
               axis=0, ascending=True,
               inplace=True)
price_yahoo


open_price  high_price  low_price  close_price  \
stock_isin   date                                                         
AN8068571086 2017-02-23   62.592112   63.196190  62.484539    62.484539   
             2017-02-24   62.550735   62.964485  62.170085    62.790710   
             2017-02-27   63.146519   63.154796  62.476246    63.113419   
             2017-02-28   63.072062   63.535460  62.807262    62.807262   
             2017-03-01   62.757614   64.089890  62.757614    64.007141   
...                             ...         ...        ...          ...   
ZM0000000037 2022-02-17    0.910000    0.910000   0.910000     0.910000   
             2022-02-18    0.950000    0.950000   0.950000     0.950000   
             2022-02-21    0.950000    0.950000   0.950000     0.950000   
             2022-02-22    0.860000    0.900000   0.860000     0.900000   
             2022-02-23    0.830000    0.900000   0.830000     0.900000   

                         trading_volume  dividends  stock_splits  \
stock_isin   date                                                  
AN8068571086 2017-02-23          1068.0        0.0           0.0   
             2017-02-24          4463.0        0.0           0.0   
             2017-02-27          1020.0        0.0           0.0   
             2017-02-28           664.0        0.0           0.0   
             2017-03-01          3064.0        0.0           0.0   
...                                 ...        ...           ...   
ZM0000000037 2022-02-17          1972.0        0.0           0.0   
             2022-02-18            20.0        0.0           0.0   
             2022-02-21             0.0        0.0           0.0   
             2022-02-22          4877.0        0.0           0.0   
             2022-02-23          1110.0        0.0           0.0   

                         adjusted_close_price  close_price_plus_dividends  
stock_isin   date                                                          
AN8068571086 2017-02-23                   NaN                   62.484539  
             2017-02-24                   NaN                   62.790710  
             2017-02-27                   NaN                   63.113419  
             2017-02-28                   NaN                   62.807262  
             2017-03-01                   NaN                   64.007141  
...                                       ...                         ...  
ZM0000000037 2022-02-17                   NaN                    0.910000  
             2022-02-18                   NaN                    0.950000  
             2022-02-21                   NaN                    0.950000  
             2022-02-22                   NaN                    0.900000  
             2022-02-23                   NaN                    0.900000  

[1688166 rows x 9 columns]

In [32]:
price_yahoo['pct_change'] = price_yahoo['close_price'].pct_change() + 1
price_yahoo.head()


open_price  high_price  low_price  close_price  \
stock_isin   date                                                         
AN8068571086 2017-02-23   62.592112   63.196190  62.484539    62.484539   
             2017-02-24   62.550735   62.964485  62.170085    62.790710   
             2017-02-27   63.146519   63.154796  62.476246    63.113419   
             2017-02-28   63.072062   63.535460  62.807262    62.807262   
             2017-03-01   62.757614   64.089890  62.757614    64.007141   

                         trading_volume  dividends  stock_splits  \
stock_isin   date                                                  
AN8068571086 2017-02-23          1068.0        0.0           0.0   
             2017-02-24          4463.0        0.0           0.0   
             2017-02-27          1020.0        0.0           0.0   
             2017-02-28           664.0        0.0           0.0   
             2017-03-01          3064.0        0.0           0.0   

                         adjusted_close_price  close_price_plus_dividends  \
stock_isin   date                                                           
AN8068571086 2017-02-23                   NaN                   62.484539   
             2017-02-24                   NaN                   62.790710   
             2017-02-27                   NaN                   63.113419   
             2017-02-28                   NaN                   62.807262   
             2017-03-01                   NaN                   64.007141   

                         pct_change  
stock_isin   date                    
AN8068571086 2017-02-23         NaN  
             2017-02-24    1.004900  
             2017-02-27    1.005139  
             2017-02-28    0.995149  
             2017-03-01    1.019104